To run, need to connect to fatboy:
 ssh -L 51433:fatboy.phys.washington.edu:1433 gateway.astro.washington.edu

If non-astro user, use simsuser@gateway.astro.washington.edu

setup sims_catalogs -t $USER -t sims

setup sims_catUtils -t $USER -t sims

In [1]:
%matplotlib inline
import numpy as np
import healpy as hp
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates import ICRS, Galactic
from lsst.sims.catalogs.db import CatalogDBObject
from lsst.sims.catUtils.baseCatalogModels import *
from lsst.sims.catUtils.exampleCatalogDefinitions import *
from lsst.sims.utils import ObservationMetaData

/Users/yoachim/lsst/DarwinX86/miniconda2/3.19.0.lsst4/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/Users/yoachim/gitRepos/sims_catalogs/python/lsst/sims/catalogs/db/dbConnection.py:439: UserWarning: Duplicate object type id 25 specified: 
Output object ids may not be unique.
This may not be a problem if you do not want globally unique id values
  'want globally unique id values')
/Users/yoachim/gitRepos/sims_catalogs/python/lsst/sims/catalogs/db/dbConnection.py:439: UserWarning: Duplicate object type id 40 specified: 
Output object ids may not be unique.
This may not be a problem if you do not want globally unique id values
  'want globally unique id values')


In [2]:
# Parameters we need to set

# Magnitudes of gray cloud extinction one should be able to see through
max_extinction = 1.0

# Precision of the measured extinction
extinction_precision = 0.2

# Dynamic range of all-sky camera. Min=observed saturation mag. 
# Max=faint end of catalog (or where Gaussian error no longer true)
mag_range = [7, 12]

# Angular resolution requirement, diameter
ang_resolution = 1. #  Degrees

# Time resolution
time_resolution = 30. #  Seconds

# Seeing requirement

# Sky brightness requirement

# Airmass requrement
airmass_limit = 2.0

# Atmospheric extinction
kAtm = 0.15

# filter to use
filtername = 'g'

In [3]:
# 5-sigma limiting depth of a point source at zenith on a clear night for the camera.
m5 = 10.

In [4]:
# Coordinates at the galactic pole
c = SkyCoord(Galactic, l=0.*u.degree , b=-90*u.degree)

In [5]:
c.icrs.ra.deg, c.icrs.dec

(12.85947789477605, <Latitude -27.128252414968006 deg>)

In [6]:
# Radius to query, in degrees
boundLength = ang_resolution/2.

colnames = ['raJ2000', 'decJ2000', 'umag', 'gmag', 'rmag', 'imag', 'zmag', 'ymag']
constraint = 'gmag < %f' % (np.max(mag_range)+ max_extinction + airmass_limit*kAtm)

# dbobj = CatalogDBObject.from_objid('allstars')
dbobj = CatalogDBObject.from_objid('brightstars')

obs_metadata = ObservationMetaData(boundType='circle',
                                   pointingRA=c.icrs.ra.deg,
                                   pointingDec=c.icrs.dec.deg,
                                   boundLength=boundLength, mjd=5700)

t = dbobj.getCatalog('ref_catalog_star', obs_metadata=obs_metadata)

stars = t.db_obj.query_columns(colnames=colnames, obs_metadata=obs_metadata,
                               constraint=constraint, limit=1e6, chunk_size=None)
stars = [chunk for chunk in stars][0]


/Users/yoachim/lsst/DarwinX86/sqlalchemy/1.0.8.lsst3+2/lib/python/SQLAlchemy-1.0.8-py2.7-macosx-10.5-x86_64.egg/sqlalchemy/sql/elements.py:3779: SAWarning: Textual SQL expression 'gmag < 13.300000' should be explicitly declared as text('gmag < 13.300000') (this warning may be suppressed after 10 occurrences)
  {"expr": util.ellipses_string(element)})


In [7]:
print 'found %i stars within %f degrees of the galactic pole' % (stars.size, ang_resolution/2.)

found 69 stars within 0.500000 degrees of the galactic pole


In [8]:
observed_mags = stars[filtername+'mag'] + max_extinction + airmass_limit*kAtm
good = np.where( (observed_mags > np.min(mag_range)) & (observed_mags < np.max(mag_range)))
observed_mags = observed_mags[good]
print 'Able to measure %i stars though %.1f mags of cloud extinction' % (observed_mags.size, max_extinction)

Able to measure 14 stars though 1.0 mags of cloud extinction


Note, I'm assuming cloud extinction and atmosphere extinction are the only relevant things, i.e., that the clouds are not adding any additional sky brightness to the background which would also lower the SNR on the stellar photometry.

In [9]:
snr =  5.*10.**(-0.4*(observed_mags-m5))
mag_uncertanties = 1./snr
# If each star is an independent measure of the zeropoint, the final 
# SNR on the zeropoint will be the combination of all the individual star SNRs.
final_snr = np.sqrt(np.sum(snr**2))

In [10]:
zp_uncertainty = 1./final_snr
print 'final zeropoint uncertainty = %f mags' % zp_uncertainty

final zeropoint uncertainty = 0.139620 mags
